In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import time

In [2]:
# Crear directorio para almacenar imágenes
dataset_path = "dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

In [3]:
# Configuración inicial
video = cv2.VideoCapture(0)
personas = ["Ichu", "Cami", "Saul", "Avl"]  # Lista de personas
imagenes_por_persona = 200  # Número de imágenes a capturar por persona
tiempo_entre_personas = 10  # Tiempo en segundos entre capturas de diferentes personas

In [ ]:
# Captura de imágenes
print("Iniciando captura de imágenes...")
for idx, persona in enumerate(personas):
    print(f"Prepárate para capturar imágenes de: {persona}.")
    print(f"Tienes {tiempo_entre_personas} segundos para enfocar la cámara en {persona}.")
    time.sleep(tiempo_entre_personas)  # Esperar antes de comenzar la captura

    print(f"Capturando imágenes para: {persona}. Presiona 'q' para salir.")
    persona_path = os.path.join(dataset_path, persona)
    if not os.path.exists(persona_path):
        os.makedirs(persona_path)
    
    contador = 0
    while contador < imagenes_por_persona:
        ret, frame = video.read()
        if not ret:
            break

        # Detección de rostros
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        rostros = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(100, 100))

        for x, y, w, h in rostros:
            rostro = gray[y:y+h, x:x+w]
            rostro = cv2.resize(rostro, (100, 100))
            file_name = os.path.join(persona_path, f"{persona}_{contador}.jpg")
            cv2.imwrite(file_name, rostro)
            contador += 1

            # Dibujar rectángulo
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"{persona} - {contador}/{imagenes_por_persona}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        cv2.imshow("Capturando rostros", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    print(f"Imágenes capturadas para {persona}: {contador}/{imagenes_por_persona}")

video.release()
cv2.destroyAllWindows()

# Entrenamiento del modelo
print("Iniciando entrenamiento de la red neuronal...")

# Cargar las imágenes
X = []
y = []
etiquetas = {persona: idx for idx, persona in enumerate(personas)}

for persona, etiqueta in etiquetas.items():
    persona_path = os.path.join(dataset_path, persona)
    for archivo in os.listdir(persona_path):
        img = cv2.imread(os.path.join(persona_path, archivo), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(etiqueta)

X = np.array(X).reshape(-1, 100, 100, 1) / 255.0  # Normalización
y = to_categorical(y, num_classes=len(personas))

# Crear el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(personas), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=16)

# Guardar el modelo
model.save("modelo_rostros.h5")
print("Modelo entrenado y guardado como 'modelo_rostros.h5'.")
